## Flaconi Web Scrapping

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
pd.set_option('display.max_rows', None)
from IPython.display import display, HTML
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from selenium import webdriver
driver = webdriver.Chrome()
from selenium.webdriver.common.by import By

from textblob_de import TextBlobDE
from datetime import datetime

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 123.0.6312.124 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0120A813+48355]
	(No symbol) [0x0119C4B1]
	(No symbol) [0x010A5358]
	(No symbol) [0x010C61AC]
	(No symbol) [0x010C1EF3]
	(No symbol) [0x010C0579]
	(No symbol) [0x010F0C55]
	(No symbol) [0x010F093C]
	(No symbol) [0x010EA536]
	(No symbol) [0x010C82DC]
	(No symbol) [0x010C93DD]
	GetHandleVerifier [0x0146AABD+2539405]
	GetHandleVerifier [0x014AA78F+2800735]
	GetHandleVerifier [0x014A456C+2775612]
	GetHandleVerifier [0x012951E0+616112]
	(No symbol) [0x011A5F8C]
	(No symbol) [0x011A2328]
	(No symbol) [0x011A240B]
	(No symbol) [0x01194FF7]
	BaseThreadInitThunk [0x75CE7BA9+25]
	RtlInitializeExceptionChain [0x775BBE3B+107]
	RtlClearBits [0x775BBDBF+191]


## get the page links (need to change the Input)

In [2]:
%%time
# get the pag links
htmls = []

for page in range(0,559): #if there are 3 pages, then input 3 
        
    page = page * 24
    url = 'https://www.flaconi.de/pflege/?offset={}'
    htmls.append(url.format(page))
    
         
len(htmls)

CPU times: total: 0 ns
Wall time: 0 ns


559

## get the product links.  

In [3]:
%%time
# get the product links

productlinks = []

for html in htmls:
    

    r = requests.get(html)
    soup = BeautifulSoup(r.content, 'lxml')
    
    for item in soup.find_all('div', class_="ProductItemVerticalstyle__Wrapper-sc-1ihgv7p-19 efiUOb"):
        for link in item.find_all('a', href=True):
            productlinks.append('https://www.flaconi.de' + link['href'])
    
   
len(productlinks)

CPU times: total: 59 s
Wall time: 15min 58s


13384

In [18]:
def brand(soup):
    try:
        brand = soup.find('h1', class_='BrandProductNameAndTypestyle__Wrapper-sc-117vbmi-0 gwyTij').find('a')['title']
    except (AttributeError, TypeError): 
        return 
    
    return brand


def name(soup):
    try:
        name = soup.find('span', class_="Headlines__H2-sc-2f0fsw-1 hEMkhZ").text
    except AttributeError: 
        return 
    
    return name


def id(row):
    try:
        id = row['data-product-sku']

    
    except AttributeError: 
        return 
    
    return id

def price1(row):
    try:
        price1 = row.find('div', class_ ='ProductPricestyle__PriceValue-sc-1flcbsi-1').text.replace('€','').replace('\xa0','').replace(",", ".")
    except AttributeError: 
        return 
    
    return price1

def price2(row):
    try:
        price2 = row.find('div', class_ ='ProductPricestyle__Discount-sc-1flcbsi-2 bwoLlr').find('label').text.replace('€','').replace('\xa0','').replace(",", ".").replace("UVP", "").replace(" ", "")
    except AttributeError: 
        return 
    
    return price2


def size(row):
    try:
        size = row.find('div', class_ ='ProductTitlestyle__Value-sc-18jmo3p-1 flAgYG').text
    except AttributeError:
        return
    
    return size


def discount(row):
    try:
        discount1 = row.find('div', class_ ='ProductPricestyle__Discount-sc-1flcbsi-2 bwoLlr').text
        discount =  re.split("\s", discount1)[0].replace('%','').replace('-','')
    
    except AttributeError: 
        return 
    
    return discount


def reviews(soup):
    try:
        reviews = re.search('"ratingCount":([\d\.]+)', soup.find_all('script', type="application/ld+json")[1].text).group(1)
    except AttributeError:
        return 0
    
    return reviews


def ratings(soup):
    try:
        ratings = re.search('"ratingValue":([\d\.]+)', soup.find_all('script', type="application/ld+json")[1].text).group(1)
    except AttributeError:
        return 0
    
    return ratings

In [20]:
%%time

df  = pd.DataFrame(columns=['id','EAN','brand', 'name','discount', 'price1', 'price2','size', "reviews","ratings",'sorting','links'])


for i, link in enumerate(productlinks1):

    driver.get(link)
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    for row in soup.find_all(['div', 'label'],  {"class":["ProductVariantSelectorstyle__WrapperDiv-sc-15rhp37-1", "ProductVariantSelectorstyle__WrapperLabel-sc-15rhp37-0"]}):
      
        link_ean = link + '#sku=' + str(id(row))
        driver.get(link_ean)
        soup = BeautifulSoup(driver.page_source, 'lxml') 
        EAN = soup.find('img', class_ = 'ProductPreviewSliderstyle__Image-sc-1t0tp5v-2 grpdtf')['src'].split(".jpg")[0].split("-")[-1]
            
        product = {

        
        'id': [id(row)],
        'EAN': [EAN],
        'brand': [brand(soup)],
        'name': [name(soup)],
        'discount':None,
        'price1': [price1(row)],
        'price2': [price2(row)],
        'size': [size(row)],
        "reviews": [reviews(soup)],
        "ratings": [ratings(soup)],
        'sorting': [i],
        'links' : [link],

        }

        product = pd.DataFrame(product)

        
        #df.append(product)
        df = df.append(product)

    df.append(df)
    df = df.reset_index().drop('index',  axis=1)


df.to_csv('raw.csv', index=False)
df.shape

CPU times: total: 44min 50s
Wall time: 6h 54min 49s


(13479, 12)

In [1]:
display(HTML("<div style='height: 500px'>" + df.style.render() + "</div>"))
   

## Data cleaning

In [19]:
# convert the variables below to float
df[["price1","price2",'ratings','reviews','sorting','discount']] = df[["price1","price2",'ratings','reviews','sorting','discount']].astype(float)


# convert the sizes to float
for i in range(0,df.shape[0]):
    try:
        df.at[i,'size'] = pd.to_numeric(df.loc[i]['size'], errors='ignore', downcast='float')
    except ValueError:
        pass

#If price2 is empty, then price2 = price1
for i in range(0, df.shape[0]):
    if pd.isnull(df.loc[i]['price2']):
        df.at[i,'price2'] = df.loc[i]['price1']


#df['scale polarity (reviews*polarity)'] = df['reviews']*df['polarity']
df['price2 minus price1'] = df['price2'] - df['price1']


#price per 100 ml
df['price1 per 100ml'] = None
df['price2 per 100ml'] = None

for i in range(0,df.shape[0]):
    try:
        df.at[i, 'price1 per 100ml']  = (df['price1'].iloc[i]/df['size'].iloc[i])*100
    except TypeError:    
        df.at[i,'price1 per 100ml'] = None
    try:
        df.at[i, 'price2 per 100ml']  = (df['price2'].iloc[i]/df['size'].iloc[i])*100
    except TypeError:    
        df.at[i,'price2 per 100ml'] = None
        

# create a new variable 'brand + name' 
df['brand + name'] = df['brand'] + ' ' + df['name'] 
for i in range(0,df.shape[0]):
    df.at[i,'brand + name'] = str(df['brand + name'].iloc[i]).title().replace('  ',' ')
        

## flaconi Customers Comments Scrapping

## Sentiment Analysis by Textblod

## delivery day

## label the categories

In [20]:


df['p_category'] = None ### need to change
df['pp_category'] = 'Pflege '### need to change
df['main_category'] = 'Pflege'### need to change
df['source'] = 'flaconi'
df.head()

,id,EAN,brand,name,brand + name,size,discount,price1,price2,price2 minus price1,...,price2 per 100ml,reviews,ratings,sorting,p_category,pp_category,main_category,source,links,deliveryTime
0,80049483-50,817777003819,NIA-114,Tighten&Lift Tightening Face Serum,Nia-114 Tighten&Lift Tightening Face Serum,50.0,NaN,81.95,81.95,0.00,...,163.9,0.0,0.00000,0.0,None,Pflege,Pflege,flaconi,https://www.flaconi.de/pflege/strivectin/tight...,2
1,80050333-50,4016083059718,Hildegard Braukmann,Exquisit Creme Bleue,Hildegard Braukmann Exquisit Creme Bleue,50.0,NaN,18.95,22.00,3.05,...,44.0,3.0,4.66667,1.0,None,Pflege,Pflege,flaconi,https://www.flaconi.de/pflege/hildegard-braukm...,2
2,80032401-5,8809403815233,WHAMISA,Organic Flowers Water Cream,Whamisa Organic Flowers Water Cream,5.0,NaN,8.95,8.95,0.00,...,179.0,5.0,5.00000,2.0,None,Pflege,Pflege,flaconi,https://www.flaconi.de/pflege/whamisa/organic-...,2
3,80032401-51,8809403814960,WHAMISA,Organic Flowers Water Cream,Whamisa Organic Flowers Water Cream,51.0,NaN,45.95,52.00,6.05,...,101.960784,5.0,5.00000,2.0,None,Pflege,Pflege,flaconi,https://www.flaconi.de/pflege/whamisa/organic-...,2
4,80024638-150,3605975080865,Kiehl's,Ultra Facial Oil-Free Cleanser,Kiehl'S Ultra Facial Oil-Free Cleanser,150.0,NaN,16.95,16.95,0.00,...,11.3,5.0,4.60000,3.0,None,Pflege,Pflege,flaconi,https://www.flaconi.de/pflege/kiehl-s/ultra-fa...,2


In [21]:
#df = df[['id', 'EAN', 'brand', 'name', 'size', 'discount', 'price1', 'price2', 'price2 minus price1', 'price1 per 100ml','price2 per 100ml','reviews', 'ratings', 'sorting', 'p_category', 'pp_category', 'main_category', 'sourse', 'links', 'comments', 'polarity','scale polarity (reviews*polarity)','deliveryTime']]

df = df[['id', 'EAN', 'brand', 'name', 'brand + name', 'size', 'discount', 'price1', 'price2', 'price2 minus price1', 'price1 per 100ml','price2 per 100ml','reviews', 'ratings', 'sorting', 'p_category', 'pp_category', 'main_category', 'source', 'links', 'deliveryTime']]
df.shape

(17020, 21)

In [22]:
df.to_csv('pflege.csv', index=False)

### Combine all of datasets

In [194]:
a = pd.read_csv("Gesichtspflege.csv")
b = pd.read_csv("Waschen & Pflegen_sp.csv")
c = pd.read_csv("Styling.csv")
d = pd.read_csv("Styling_sp.csv")
e = pd.read_csv("For Men.csv")
#f = pd.read_csv("Haarwachs & Gel.csv")
#g = pd.read_csv("Anti-Schuppen.csv")
#h = pd.read_csv("haarmaske.csv")




flaconi_raw= pd.concat([a, b, c, d ,e])

flaconi_raw.to_csv('Haare.csv', index=False)

In [13]:
len(productlinks[2914:])

10470

In [ ]:
https://www.flaconi.de/pflege/happybrush/rotierend/happybrush-rotierend-r2-elektrische-zahnbuerste.html